# 🧠 MINI v1 Agent: Module A (Major Depressive Episode)
This notebook tests the ReAct agent's ability to conduct MINI Module A using LangChain + OpenAI.

In [2]:
!pip install langchain openai python-dotenv pandas --quiet

## ✅ 1. Load API key from `.env`

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  # ✅ This is the name of the variable, not the key itself

if not api_key:
    raise ValueError("⚠️ OPENAI_API_KEY not found in your .env file.")


## ✅ 2. Set up LangChain ReAct Agent

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools import Tool
from langchain.prompts import PromptTemplate

# Load model
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key=api_key
)

# Define a custom tool
def interpret_response(input: str) -> str:
    if "yes" in input.lower():
        return "Positive response"
    elif "no" in input.lower():
        return "Negative response"
    return "Ambiguous"

tool = Tool.from_function(
    func=interpret_response,
    name="InterpretResponse",
    description="Classifies patient response as positive, negative, or ambiguous."
)

prompt_template = PromptTemplate.from_template(
    """Answer the following questions as a helpful psychiatric interview assistant. 
You should use tools if they are relevant.

Available tools:
{tools}

You must use the format:

Question: {input}
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""
)

# Create agent + executor
agent = create_react_agent(llm=llm, tools=[tool], prompt=prompt_template)
executor = AgentExecutor(agent=agent, tools=[tool], verbose=True)


## ✅ 3. Load MINI Module A Questions

In [18]:
module_a_questions = [
    "During the past two weeks, have you been consistently depressed or down most of the day, nearly every day?",
    "Have you lost interest or pleasure in most things you usually enjoy?",
    "Have you experienced significant weight change or appetite disturbance?",
    "Have you had trouble sleeping or sleeping too much?",
    "Have you felt slowed down or restless?",
    "Have you felt worthless or excessively guilty?",
    "Have you had trouble concentrating or making decisions?",
    "Have you had thoughts that you would be better off dead or hurting yourself?"
]

## ✅ 4. Define Judge Agent

In [19]:
def judge_check(response: str, interpretation: str):
    red_flags = ["suicide", "kill", "hurt myself", "end my life"]
    if any(flag in response.lower() for flag in red_flags):
        return "⚠️ Red Flag — Escalate to clinician"
    if ("yes" in response.lower() and "negative" in interpretation.lower()) or        ("no" in response.lower() and "positive" in interpretation.lower()):
        return "⚠️ Discrepancy — Flag for review"
    return "✅ Aligned"


## ✅ 5. Run Interactive Test Loop

In [22]:
for question in module_a_questions:
    print("\n🧠 MINI Agent:", question)
    response = input("Patient: ")
prompt = f"Question: {question}\nPatient says: {response}"

result = executor.invoke({"input": prompt})
interpretation = result["output"]

print("🤖 Agent Interpretation:", interpretation)
print("🧪 Judge QA:", judge_check(response, interpretation))


🧠 MINI Agent: During the past two weeks, have you been consistently depressed or down most of the day, nearly every day?


Patient:  Yes



🧠 MINI Agent: Have you lost interest or pleasure in most things you usually enjoy?


Patient:  No



🧠 MINI Agent: Have you experienced significant weight change or appetite disturbance?


...


🧠 MINI Agent: Have you had trouble concentrating or making decisions?


Patient:  no



🧠 MINI Agent: Have you had thoughts that you would be better off dead or hurting yourself?


Patient:  no




> Entering new AgentExecutor chain...
Thought: The patient's response seems clear, but I should use the InterpretResponse tool to confirm.
Action: InterpretResponse
The InterpretResponse tool confirmed my initial understanding of the patient's response.
Final Answer: The patient has not had thoughts of being better off dead or hurting themselves.

> Finished chain.
🤖 Agent Interpretation: The patient has not had thoughts of being better off dead or hurting themselves.
🧪 Judge QA: ✅ Aligned
